In [1]:
import numpy as np
import pandas as pd

In [2]:
# path
inputFolder = r"/Volumes/Mingyue_Plu/deltaN1E4/Mingyue_analysis_2/smapling/undersampling"
folderseparator = "/"
X = []
y = []
for i in range(121,131):#131
    myFFN = "sim%d_undersamp_x.csv" % (i)
    filenamex = inputFolder + folderseparator + myFFN
    df = pd.read_csv(filenamex)
    XX = df.iloc[:,0:60]
    X.append(XX)

    myTFN = "sim%d_undersamp_y.csv" % (i)
    filenamey = inputFolder + folderseparator + myTFN
    dt = pd.read_csv(filenamey)
    yy = dt.iloc[:,0]
    y.append(yy)

X = np.concatenate(X)
y = np.concatenate(y)
print('feature shape',X.shape)
print('target shape', y.shape)

feature shape (23160, 60)
target shape (23160,)


In [3]:
features_select = [9, 0, 2, 22, 50, 48, 11, 3, 52, 38] #[50,49,17,31,35,43,22,0,1,16]

X2 = X[:,features_select]
X2.shape

from sklearn.model_selection import train_test_split
rseed = 0
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size = 0.3,stratify = y,random_state = rseed)
print(X_train.shape)

(16212, 10)


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate
scaling = StandardScaler()
model =  RandomForestClassifier(random_state=rseed)
pipeline = make_pipeline(scaling,model)

In [5]:
cv = StratifiedKFold(n_splits=5)

n_estimators = [200, 4000,600 ]
max_features = ['log2']
max_depth = [3,5,7]
min_samples_split = [2,3,4]
min_samples_leaf = [1,2,3]
param_grid = {'randomforestclassifier__n_estimators': n_estimators,
              'randomforestclassifier__max_features': max_features, 
              'randomforestclassifier__max_depth': max_depth,
              'randomforestclassifier__min_samples_split': min_samples_split,
              'randomforestclassifier__min_samples_leaf': min_samples_leaf
              
               }

gs = RandomizedSearchCV(estimator=pipeline, 
                  param_distributions=param_grid, 
                  scoring='roc_auc', 
                  refit=True,
                  cv=cv)

gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.7766248185157079
{'randomforestclassifier__n_estimators': 600, 'randomforestclassifier__min_samples_split': 3, 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__max_features': 'log2', 'randomforestclassifier__max_depth': 7}


In [6]:
print('Training set roc-auc: ' + str(gs.score(X_train, y_train)))
print('Testing set roc-auc: ' + str(gs.score(X_test, y_test)))

Training set roc-auc: 0.8223871136231299
Testing set roc-auc: 0.7774770635550299


In [7]:
from sklearn.metrics import confusion_matrix
inputFolder = r"/Volumes/Mingyue_Plu/deltaN1E4/Mingyue_analysis_2"
folderseparator = "/"
i=121
myFN = 'sim%d_d2min_tho015_int1.csv' % (i)
filename = inputFolder + folderseparator + myFN
df = pd.read_csv(filename,header = None)
Xnew = df.iloc[:,0:60]
Xnew = Xnew.values
dy = pd.read_csv('d2min02.csv',header = None)
ynew = dy.iloc[:,0]
ynew = ynew.values
syn_pred = gs.best_estimator_.predict(Xnew)

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(ynew, syn_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

from sklearn.metrics import roc_auc_score
roc_auc_score(ynew, syn_pred, average=None)

ValueError: X has 60 features, but StandardScaler is expecting 10 features as input.